In [ ]:
# Standard Libraries of Python
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

# Libraries made for this Proyect
from database.clean_database import database
from data_analisys import data_functions

In [ ]:
# Main Object
numbers = range(1,51)

class Analisis:
    def __init__(self):
        self.df = pd.read_parquet('database/db.parquet')
        self.lottery = self.df.copy()
        self.lottery['Dates'] = self.lottery['Dates'].dt.year
        self.lottery = self.lottery.drop(columns=['Sorteo','Star_1','Star_2'])
    
    def __transformation_into_columns(self,row):
        for draw in range(1,6):
            if not np.isnan(self.new_lottery.loc[row.Dates,row[f'Nro{draw}']]):
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] += 1
            else:
                self.new_lottery.loc[row.Dates,row[f'Nro{draw}']] = 1
    
    def __numbers_boolean(self):
        self.skips_winners_bool = pd.DataFrame(False, columns=[int(i) for i in numbers],
            index=range(len(self.df)))
        row_0 = pd.DataFrame(columns=[int(i) for i in numbers],
            index=[0]).fillna(True)
        for e in range(1,6):
            col_name = f"Nro{e}"
            self.skips_winners_bool = self.skips_winners_bool | (self.df[col_name].to_numpy()[:, None] == numbers)
        self.boolean_df = pd.concat([row_0, self.skips_winners_bool]).reset_index(drop=True)

    def __total_average_hits(self,is_star=False,aprox=False):
        divide = 2 if is_star else 5
        self.average = self.hits.apply(lambda hits: hits / len(self.df) / divide).iloc[0]
        if aprox:
            return Decimal(self.average.sum()) / Decimal(int(50)) + Decimal(0.001)
        else:
            return Decimal(self.average.sum()) / Decimal(int(50))

    def __natural_rotation(self,index_start,index_end,is_star=False,aprox=False):
        self.__total_average_hits(is_star=is_star,aprox=aprox)
        rotation = pd.DataFrame({'hits': self.hits.iloc[0],
            'average_numbers': self.average,
            'average': self.__total_average_hits(is_star=is_star,aprox=aprox),
            'hits_needed': self.m_hits(is_star=is_star,aprox=aprox)},
            index=range(index_start,index_end+1))
        rotation['difference'] = rotation['hits'] - rotation['hits_needed']
        return rotation
        
    def apply_transformation(self):
        self.new_lottery = pd.DataFrame(columns=[i for i in numbers],
            index=np.arange(self.lottery['Dates'].iloc[0],self.lottery['Dates'].iloc[-1]+1))
        self.lottery.apply(self.__transformation_into_columns,axis=1)
        self.new_lottery.fillna(0,inplace=True)

        self.hits = self.new_lottery.sum().to_frame().rename(columns={0: 'hits'}).T.astype('int32')
        self.mean = self.new_lottery.mean().to_frame().rename(columns={0: 'mean'}).T.astype('float32')
        self.median = self.new_lottery.median().to_frame().rename(columns={0: 'median'}).T.astype('float32')

    def m_hits(self,is_star=False,aprox=False):
        min_hits = self.__total_average_hits(is_star=is_star,aprox=aprox)
        return min_hits * Decimal(int(self.hits.iloc[0,0])) / Decimal(float(self.average.iat[0]))

    def get_natural_rotations(self,is_star=False):
        self.exact_rotation = self.__natural_rotation(1,50,is_star=is_star,aprox=False)
        self.aprox_rotation = self.__natural_rotation(1,50,is_star=is_star,aprox=True)

    def count_skips(self):
        self.__numbers_boolean()
        dicts = {draw: {key: [] for key in numbers} for draw in range(1,len(self.boolean_df))}
        for draw in range(1,len(self.boolean_df)):
            df = self.boolean_df.loc[:draw]
            counts = {key: 0 for key in numbers}
            for column in df:
                counter = 0
                for value in reversed(df[column]):
                    if not value:
                        counter += 1
                    else:
                        counts[column] = counter
                        break
            dicts[draw].update(counts)
        self.counts = pd.DataFrame.from_dict(dicts,orient='index')
    
    def numbers_clasification(self):
        best_numbers = self.aprox_rotation.loc[self.aprox_rotation['hits'] > self.aprox_rotation['hits_needed']].index.to_numpy()
        normal_numbers = self.exact_rotation.loc[(self.exact_rotation['hits'] > self.exact_rotation['hits_needed']) & ~(self.exact_rotation.index.isin(best_numbers))].index.to_numpy()
        rotation_info = {number: 0 for number in numbers}
        for number in best_numbers:
            rotation_info[number] = 2
        for number in normal_numbers:
            rotation_info[number] = 1

In [ ]:
euromillones = Analisis()
euromillones.apply_transformation()
euromillones.get_natural_rotations()
euromillones.count_skips()

In [ ]:
current_year = euromillones.lottery.iloc[-1]
year_criteria = {key: [] for key in numbers}

for number in numbers:
    x = euromillones.new_lottery.at[current_year, number]
    median_half = euromillones.median.iat['median',number] / 2

    if euromillones.mean.iat['average',number] != 0 and not np.isnan(
        euromillones.mean.iat['average',number]
        ):
        if x == 0 or x <= median_half:
            y = round((1 - ((euromillones.median.iat['median',number] * 100) / 
                euromillones.mean.iat['average',number]) / 100), 2)
        else:
            x_percentage = round(
                (x * 100 / euromillones.mean.iat['average',number]) / 100, 2
                )
            y = round((1 - x_percentage if x_percentage > 1 else 1 - x_percentage), 2)
    elif y == 0:
        y = 0.50
    else:
        y = 0

    year_criteria[number] = float(y)

year_criteria = pd.DataFrame.from_dict(year_criteria, orient='index', columns=['year_criteria'])

In [ ]:
# Finding missing numbers
best_numbers = df_aprox.loc[df_aprox['Hits'] > df_aprox['Hits_Needed']].index.to_numpy()
normal_numbers = df_exact.loc[(df_exact['Hits'] > df_exact['Hits_Needed']) & ~(df_exact.index.isin(best_numbers))].index.to_numpy()

# Rotation information
rotation_info = {number: 0 for number in total_numbers}
for number in best_numbers:
    rotation_info[number] = 2
for number in normal_numbers:
    rotation_info[number] = 1

# Finding current hits needed
current_hits_needed = data_functions.minimal_hits(db, total_hits, total_numbers, numbers_average, aprox=True)

# Finding rotation hit
rotation_hit = next((draw for draw in range(len(db['Sorteo'])+1, len(db['Sorteo'])+12) if draw * data_functions.minimal_hits(db, total_hits, total_numbers, numbers_average, aprox=True) / len(db) > int(current_hits_needed) + 1), None)

# Rotation criteria
rotation_criteria = {}
for i in total_numbers:
    category = rotation_info[i]
    diff_exact = df_exact.at[i, 'Difference']
    diff_aprox = df_aprox.at[i, 'Difference']
    hits_exact = df_exact.at[i, 'Hits']
    if category == 1 and diff_exact > 0.50:
        rotation_criteria[i] = 0.50
    elif category == 1 and -1 < diff_exact <= 0.40:
        x = (abs(diff_exact) + (Decimal('0.5') * Decimal('100')) / Decimal('0.50')) / Decimal('100') + Decimal('0.50')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    elif category == 2 and hits_exact > current_hits_needed:
        rotation_criteria[i] = 1
    elif category == 2 and diff_aprox > -1 and rotation_hit is not None and rotation_hit - len(db) < 4 and len(best_numbers) < 17:
        x = (abs(diff_exact) * Decimal('100')) / Decimal('1.00') / Decimal('100') + Decimal('1')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    else:
        rotation_criteria[i] = 0.25

rotation_criteria = pd.DataFrame.from_dict(rotation_criteria, orient='index', columns=['rotation_criteria'])